In [1]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import webbrowser
import time
import string
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from git_ignore.config import *

In [3]:
auth = tweepy.OAuthHandler(twitter_consumer_key, twitter_consumer_secret, twitter_callback_uri)
redirect_url = auth.get_authorization_url()
# print(redirect_url)

# webbrowser.open(redirect_url)




True

In [4]:
# user_pin_input = input("Enter pin value: ")

In [5]:
# auth.get_access_token(user_pin_input)
api = tweepy.API(auth)


# Get popular/recent tweets from keyword search

In [15]:
tweets = tweepy.Cursor(api.search, q='london', result_type='popular').items(3)

keyword_df = pd.DataFrame()
name = []
location = []
text = []

for tweet in tweets:
    name.append(tweet.user.name)
    location.append(tweet.user.location)
    text.append(tweet.text)

keyword_df['name'] = name
keyword_df['location'] = location
keyword_df['text'] = text

keyword_df

,name,location,text
0,joe helferty,"Belfast, Ireland.",RT @Pal_action: Arrestee support required at H...
1,Mohd Danial,"Sungai Petani Kedah, Malaysia",RT @falasteen47: BREAKING: London’s Downing St...
2,🌼🐱,,RT @falasteen47: BREAKING: London’s Downing St...


In [13]:
keyword_df

,name,location,text,result_type
0,MrBeast,Follow me for a cookie,MrBeast Burger is now available in the UK! We ...,popular
1,Jairam Ramesh,India,"My younger son, who is currently pursuing his ...",popular
2,Andy Burnham,"Leigh, Greater Manchester",Thanks for your replies. Seems things are much...,popular
3,MrBeast,Follow me for a cookie,MrBeast Burger is now available in the UK! We ...,recent
4,Jairam Ramesh,India,"My younger son, who is currently pursuing his ...",recent
5,Andy Burnham,"Leigh, Greater Manchester",Thanks for your replies. Seems things are much...,recent


In [109]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Table(
        header=dict(values=list(keyword_df.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[keyword_df.name, keyword_df.location, keyword_df.text],
                   fill_color='lavender',
                   align='left'))
    )


fig.show()

# Get tweets from keyword search

# Get location heatmap using keyword

In [13]:
keyword = 'palestine'
WORLD = 1
CANADA = 23424775

rj_trends = api.trends_place(id = CANADA)
trends = []
for trend in rj_trends[0]['trends']: 
    if trend['tweet_volume'] is not None and trend['tweet_volume'] > 10000: 
        trends.append((trend['name'], trend['tweet_volume']))

trends.sort(key=lambda x:-x[1])
trends

[('Gaza', 1550116),
 ('Hamas', 649173),
 ('The UK', 567201),
 ('taylor', 506359),
 ('harry styles', 419153),
 ('Tel Aviv', 318950),
 ('#brits2021', 168568),
 ('Iron Dome', 161632),
 ('Israelis', 156920),
 ('olivia', 156300),
 ('Middle East', 145284),
 ('Victor', 111639),
 ('Dr. Fauci', 102440),
 ('Barcelona', 101678),
 ('Liverpool', 97531),
 ('Leicester', 94963),
 ('Barca', 94942),
 ('AstraZeneca', 81200),
 ('Roberto', 74253),
 ('Rand Paul', 69329),
 ('Greenwood', 57165),
 ('Koeman', 56193),
 ('#MUNLEI', 44296),
 ('Line 5', 41010),
 ('Booked', 37812),
 ('Abel', 37150),
 ('Michigan', 36573),
 ('Matic', 34756),
 ('HAIM', 33852),
 ('Dembele', 28468),
 ('Oakland', 25238),
 ('Hamilton', 23581),
 ('Elton John', 18711),
 ('Las Vegas', 14149),
 ('Nash', 10386)]

# Trending tweets from location keyword

In [19]:
cwoeid = 23424775
woeid = 1
  
# fetching the trends
rj_trends = api.trends_place(id = cwoeid)
trends = []
for trend in rj_trends[0]['trends']: 
    if trend['tweet_volume'] is not None and trend['tweet_volume'] > 10000: 
        trends.append((trend['name'], trend['tweet_volume']))

trends.sort(key=lambda x:-x[1])
trends
# rj_trends

[('Gaza', 1550116),
 ('Hamas', 650176),
 ('The UK', 567201),
 ('taylor', 506359),
 ('harry styles', 419132),
 ('Tel Aviv', 318950),
 ('#brits2021', 168568),
 ('Iron Dome', 161864),
 ('Israelis', 156920),
 ('olivia', 156300),
 ('Middle East', 145440),
 ('Victor', 111694),
 ('Dr. Fauci', 102548),
 ('Barcelona', 101678),
 ('Liverpool', 97670),
 ('Barca', 95757),
 ('Leicester', 94963),
 ('AstraZeneca', 81279),
 ('Roberto', 74253),
 ('Rand Paul', 69427),
 ('Greenwood', 57178),
 ('Koeman', 56193),
 ('#MUNLEI', 44296),
 ('Line 5', 41034),
 ('Booked', 37812),
 ('Abel', 37186),
 ('Michigan', 36594),
 ('Matic', 34792),
 ('HAIM', 33852),
 ('Dembele', 28654),
 ('Oakland', 25238),
 ('Hamilton', 23598),
 ('Elton John', 18735),
 ('Las Vegas', 14149),
 ('Nash', 10386)]

In [83]:
import plotly.express as px
df = px.data.gapminder()
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


## Personal Tweets and Analysis

In [10]:
timeline = api.home_timeline()
for tweet in timeline:
    print(f"{tweet.user.name} said {tweet.text}")

Imran Khan said انسانیت کی تمام حدود اور عالمی قوانین پامال کرتے ہوئے خصوصاً رمضان المبارک کے دوران اسرائیلی فوجیوں کے قبلہ اول یعن… https://t.co/1KzxeLmzI2
Imran Khan said Strongly condemn Israeli Forces' attack esp during Ramazan on Palestinians in Qibla-e-Awaal, Al-Aqsa Mosque, violat… https://t.co/UpMZWC1iuP
SpaceX said Deployment of 60 Starlink satellites confirmed https://t.co/RarbrcfUml
Narendra Modi said Saddened by the demise of INA veteran Lalti Ram Ji. His courage and contributions to India’s freedom struggle will… https://t.co/E5hKmAdkXW
SpaceX said Falcon 9’s first stage has landed on the Just Read the Instructions droneship, completing this booster’s 10th launc… https://t.co/48pJZyHq1i
SpaceX said Liftoff! https://t.co/MZ4ct2Spnj
SpaceX said Watch the first tenth flight of an orbital class rocket booster → https://t.co/bJFjLCzWdK  https://t.co/MDM6RNlN4J
enzo said RT @Abz4ktt: I beg (if it is Laylatul Qadr tonight Inshallah) all of us Muslimeen in the world rn to all rais